In [12]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt

In [13]:
# Load data
images_dir = os.listdir("../input/flickr8k/flickr_data/Flickr_Data/")

images_path = '../input/flickr8k/flickr_data/Flickr_Data/Images/'
captions_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
train_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'

val_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt'
test_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'

captions = open(captions_path, 'r').read().split("\n")
x_train = open(train_path, 'r').read().split("\n")
x_val = open(val_path, 'r').read().split("\n")
x_test = open(test_path, 'r').read().split("\n")

In [ ]:
# Loading captions as values and images as key in dictionary
tokens = {}

for ix in range(len(captions)-1):
    temp = captions[ix].split("#")
    if temp[0] in tokens:
        tokens[temp[0]].append(temp[1][2:])
    else:
        tokens[temp[0]] = [temp[1][2:]]

In [ ]:
# displaying an image and captions given to it
temp = captions[10].split("#")
from IPython.display import Image, display
z = Image(filename=images_path+temp[0])
display(z)

for ix in range(len(tokens[temp[0]])):
    print(tokens[temp[0]][ix])

In [ ]:
# Creating train, test and validation dataset files with header as 'image_id' and 'captions'
train_dataset = open('flickr_8k_train_dataset.txt','wb')
train_dataset.write(b"image_id\tcaptions\n")

val_dataset = open('flickr_8k_val_dataset.txt','wb')
val_dataset.write(b"image_id\tcaptions\n")

test_dataset = open('flickr_8k_test_dataset.txt','wb')
test_dataset.write(b"image_id\tcaptions\n")

In [ ]:
# Populating the above created files for train, test and validation dataset with image ids and captions for each of these images
for img in x_train:
    if img == '':
        continue
    for capt in tokens[img]:
        caption = "<start> "+ capt + " <end>"
        train_dataset.write((img+"\t"+caption+"\n").encode())
        train_dataset.flush()
train_dataset.close()

for img in x_test:
    if img == '':
        continue
    for capt in tokens[img]:
        caption = "<start> "+ capt + " <end>"
        test_dataset.write((img+"\t"+caption+"\n").encode())
        test_dataset.flush()
test_dataset.close()

for img in x_val:
    if img == '':
        continue
    for capt in tokens[img]:
        caption = "<start> "+ capt + " <end>"
        val_dataset.write((img+"\t"+caption+"\n").encode())
        val_dataset.flush()
val_dataset.close()

In [ ]:
# Loading 50 layer Residual Network Model and getting the summary of the model
from IPython.core.display import display, HTML
display(HTML("""<a href="http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006">ResNet50 Architecture</a>"""))
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
model.summary()
# Note: For more details on ResNet50 architecture you can click on hyperlink given below

In [ ]:
# Helper function to process images
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
train_data = {}
ctr=0
for ix in x_train:
    if ix == "":
        continue
    if ctr >= 3000:
        break
    ctr+=1
    if ctr%1000==0:
        print(ctr)
    path = images_path + ix
    img = preprocessing(path)
    pred = model.predict(img).reshape(2048)
    train_data[ix] = pred

In [ ]:
train_data['2513260012_03d33305cf.jpg'].shape

In [ ]:
# opening train_encoded_images.p file and dumping it's content
with open( "train_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(train_data, pickle_f )  

In [ ]:
# Loading image and its corresponding caption into a dataframe and then storing values from dataframe into 'ds'
pd_dataset = pd.read_csv("flickr_8k_train_dataset.txt", delimiter='\t')
ds = pd_dataset.values
print(ds.shape)

In [ ]:
pd_dataset.head()

In [ ]:
# Storing all the captions from ds into a list
sentences = []
for ix in range(ds.shape[0]):
    sentences.append(ds[ix, 1])
    
print(len(sentences))

In [ ]:
# First 5 captions stored in sentences
sentences[:5]

In [ ]:
# Splitting each captions stored in 'sentences' and storing them in 'words' as list of list
words = [i.split() for i in sentences]

In [ ]:
# Creating a list of all unique words
unique = []
for i in words:
    unique.extend(i)
unique = list(set(unique))

print(len(unique))

vocab_size = len(unique)

In [ ]:
# Vectorization
word_2_indices = {val:index for index, val in enumerate(unique)}
indices_2_word = {index:val for index, val in enumerate(unique)}

In [ ]:
word_2_indices['UNK'] = 0
word_2_indices['raining'] = 8253

In [ ]:
indices_2_word[0] = 'UNK'
indices_2_word[8253] = 'raining'

In [ ]:
print(word_2_indices['<start>'])
print(indices_2_word[4011])
print(word_2_indices['<end>'])
print(indices_2_word[8051])

In [ ]:
vocab_size = len(word_2_indices.keys())
print(vocab_size)

In [ ]:
max_len = 0

for i in sentences:
    i = i.split()
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

In [ ]:
padded_sequences, subsequent_words = [], []

for ix in range(ds.shape[0]):
    partial_seqs = []
    next_words = []
    text = ds[ix, 1].split()
    text = [word_2_indices[i] for i in text]
    for i in range(1, len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')

    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    #Vectorization
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)
    
padded_sequences = np.asarray(padded_sequences)
subsequent_words = np.asarray(subsequent_words)

print(padded_sequences.shape)
print(subsequent_words.shape)

In [ ]:
print(padded_sequences[0])

In [ ]:
for ix in range(len(padded_sequences[0])):
    for iy in range(max_len):
        print(indices_2_word[padded_sequences[0][ix][iy]],)
    print("\n")

print(len(padded_sequences[0]))

In [ ]:
num_of_images = 2000

In [ ]:
captions = np.zeros([0, max_len])
next_words = np.zeros([0, vocab_size])

In [ ]:
for ix in range(num_of_images):#img_to_padded_seqs.shape[0]):
    captions = np.concatenate([captions, padded_sequences[ix]])
    next_words = np.concatenate([next_words, subsequent_words[ix]])

np.save("captions.npy", captions)
np.save("next_words.npy", next_words)

print(captions.shape)
print(next_words.shape)

In [ ]:
with open('../input/train_encoded_images.p', 'rb') as f:
    encoded_images = pickle.load(f, encoding="bytes")

In [ ]:
imgs = []

for ix in range(ds.shape[0]):
    if ds[ix, 0].encode() in encoded_images.keys():
#         print(ix, encoded_images[ds[ix, 0].encode()])
        imgs.append(list(encoded_images[ds[ix, 0].encode()]))

imgs = np.asarray(imgs)
print(imgs.shape)

In [ ]:
images = []

for ix in range(num_of_images):
    for iy in range(padded_sequences[ix].shape[0]):
        images.append(imgs[ix])
        
images = np.asarray(images)

np.save("images.npy", images)

print(images.shape)

In [ ]:
image_names = []

for ix in range(num_of_images):
    for iy in range(padded_sequences[ix].shape[0]):
        image_names.append(ds[ix, 0])
        
image_names = np.asarray(image_names)

np.save("image_names.npy", image_names)

print(len(image_names))

###  **Model**

In [14]:
captions = np.load("../input/flicker-8kimg/captions.npy")
next_words = np.load("../input/flicker-8kimg/next_words.npy")

print(captions.shape)
print(next_words.shape)

(25493, 40)
(25493, 8254)


In [15]:
images = np.load("../input/flicker-8kimg/images.npy")

print(images.shape)

(25493, 2048)


In [16]:
imag = np.load("../input/flicker-8kimg/image_names.npy")
        
print(imag.shape)

(25493,)


In [17]:
embedding_size = 128
max_len = 40
vocab_size = 8254

In [18]:
image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               262272    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 40, 128)           0         
Total params: 262,272
Trainable params: 262,272
Non-trainable params: 0
_________________________________________________________________


In [19]:
language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 128)           1056512   
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 256)           394240    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 128)           32896     
Total params: 1,483,648
Trainable params: 1,483,648
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [21]:
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

# model.load_weights("../input/model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy', recall_m, precision_m, f1_m])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_2_input (InputLayer)  (None, 40)           0                                            
__________________________________________________________________________________________________
dense_4_input (InputLayer)      (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 40, 128)      1056512     embedding_2_input[0][0]          
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 128)          262272      dense_4_input[0][0]              
__________________________________________________________________________________________________
lstm_4 (LS

In [22]:
hist = model.fit([images, captions], next_words, batch_size=512, epochs=150)

Epoch 1/150
25493/25493 [==============================] - 13s 498us/step - loss: 5.7061 - acc: 0.0714 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - f1_m: 0.0000e+00
Epoch 2/150
25493/25493 [==============================] - 10s 410us/step - loss: 5.2809 - acc: 0.0874 - recall_m: 0.0063 - precision_m: 0.0861 - f1_m: 0.0117
Epoch 3/150
25493/25493 [==============================] - 11s 432us/step - loss: 5.2237 - acc: 0.0864 - recall_m: 0.0030 - precision_m: 0.0577 - f1_m: 0.0054
Epoch 4/150
25493/25493 [==============================] - 10s 407us/step - loss: 5.1040 - acc: 0.0933 - recall_m: 0.0115 - precision_m: 0.1822 - f1_m: 0.0215
Epoch 5/150
25493/25493 [==============================] - 11s 424us/step - loss: 4.9249 - acc: 0.1160 - recall_m: 0.0300 - precision_m: 0.4617 - f1_m: 0.0559
Epoch 6/150
25493/25493 [==============================] - 11s 438us/step - loss: 4.9099 - acc: 0.1107 - recall_m: 0.0223 - precision_m: 0.4272 - f1_m: 0.0417
Epoch 7/150
25493/25493 [=========

25493/25493 [==============================] - 10s 408us/step - loss: 1.9737 - acc: 0.4419 - recall_m: 0.2658 - precision_m: 0.7595 - f1_m: 0.3935
Epoch 53/150
25493/25493 [==============================] - 10s 401us/step - loss: 1.9213 - acc: 0.4533 - recall_m: 0.2742 - precision_m: 0.7544 - f1_m: 0.4017
Epoch 54/150
25493/25493 [==============================] - 11s 418us/step - loss: 1.8788 - acc: 0.4590 - recall_m: 0.2781 - precision_m: 0.7519 - f1_m: 0.4056
Epoch 55/150
25493/25493 [==============================] - 10s 403us/step - loss: 1.8592 - acc: 0.4603 - recall_m: 0.2816 - precision_m: 0.7464 - f1_m: 0.4085
Epoch 56/150
25493/25493 [==============================] - 11s 445us/step - loss: 1.8068 - acc: 0.4706 - recall_m: 0.2911 - precision_m: 0.7552 - f1_m: 0.4198
Epoch 57/150
25493/25493 [==============================] - 11s 419us/step - loss: 1.7854 - acc: 0.4743 - recall_m: 0.2955 - precision_m: 0.7482 - f1_m: 0.4232
Epoch 58/150
25493/25493 [===========================

25493/25493 [==============================] - 11s 421us/step - loss: 0.7087 - acc: 0.7551 - recall_m: 0.6422 - precision_m: 0.8433 - f1_m: 0.7289
Epoch 104/150
25493/25493 [==============================] - 10s 402us/step - loss: 0.6907 - acc: 0.7634 - recall_m: 0.6498 - precision_m: 0.8494 - f1_m: 0.7361
Epoch 105/150
25493/25493 [==============================] - 10s 401us/step - loss: 0.6772 - acc: 0.7644 - recall_m: 0.6580 - precision_m: 0.8491 - f1_m: 0.7412
Epoch 106/150
25493/25493 [==============================] - 11s 419us/step - loss: 0.6683 - acc: 0.7685 - recall_m: 0.6619 - precision_m: 0.8505 - f1_m: 0.7443
Epoch 107/150
25493/25493 [==============================] - 11s 444us/step - loss: 0.6517 - acc: 0.7786 - recall_m: 0.6725 - precision_m: 0.8553 - f1_m: 0.7527
Epoch 108/150
25493/25493 [==============================] - 10s 409us/step - loss: 0.6326 - acc: 0.7842 - recall_m: 0.6823 - precision_m: 0.8610 - f1_m: 0.7611
Epoch 109/150
25493/25493 [=====================

In [23]:
model.save_weights("model_weights.h5")

### Predictions

In [24]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [25]:
def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [26]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

94658560/94653016 [==============================] - 1s 0us/step


In [27]:
img = "../input/flickr8k/flickr_data/Flickr_Data/Images/1453366750_6e8cf601bf.jpg"

test_img = get_encoding(resnet, img)

FileNotFoundError: [Errno 2] No such file or directory: '../input/flickr_data/Flickr_Data/Images/1453366750_6e8cf601bf.jpg'

In [ ]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

Argmax_Search = predict_captions(test_img)

In [ ]:
for imgnm in x_test[240:300]:
    print(imgnm)
    test_img = get_encoding(resnet, flickr_path + imgnm)
    Argmax_Search = predict_captions(test_img)
    z = Image(filename=flickr_path + imgnm)
    display(z)
    print(Argmax_Search)

In [ ]:
z = Image(filename=img)
display(z)

print(Argmax_Search)